# Import packages

In [1]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import precision_recall_fscore_support,roc_curve,auc,accuracy_score,classification_report, confusion_matrix
from scipy.stats.mstats import gmean
from sklearn.datasets import make_circles
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split
from sklearn.datasets import make_moons
from sklearn.preprocessing import Normalizer,MinMaxScaler,StandardScaler
import numpy as np
import os
import pandas as pd
import numpy.matlib
from numpy import random
import os
import math

from sklearn.metrics import auc
from sklearn.metrics import roc_auc_score
from sklearn.metrics import precision_recall_curve
from sklearn.metrics import f1_score

import warnings
warnings.filterwarnings("ignore")

# Hyperspheres covering methods

In [2]:
def gnerandsphere(center,d,N,r):
    random_directions = random.randn(N,d);
    s=np.sqrt(np.array(np.sum(random_directions**2,axis=1)))
    random_directions = random_directions/np.matlib.repmat(s,d,1).T; 
    random_radii =r*random.rand(N,1);
    Y = random_directions* np.matlib.repmat(random_radii,1,d);
    Y = np.matlib.repmat(center,N,1)+Y;
    return Y

In [3]:
def unionSphere(data,dataradious):
    pd = L2_dist(data,data);
    pddist =np.copy(pd);
    temp=[(v,i) for i,v in enumerate(dataradious[:,-1])]
    temp.sort(reverse=True)
    [dummy,sortid]=zip(*temp)
    sortdataradious = dataradious[sortid,:]
    #print("sortdataradious",sortdataradious )
    flag =np.zeros((len(data),1))-1
    for i in range(len(sortdataradious)):
        if flag[i] == -1:
            flag[i] = sortdataradious[i,0];
            leftindex =[]
            for j in range(i+1,len(sortdataradious)):
                if flag[j]==-1:
                    leftindex.append(j)
            
            leftunhandledid = sortdataradious[leftindex,0];
            iindex=sortdataradious[i,0]
            leftdist=[]
            for k in range(len(leftunhandledid)):
                jindex=leftunhandledid[k]
                leftdist.append(pd[iindex,jindex])
            leftdist=np.array(leftdist)
            includedset =np.argwhere(leftdist <= sortdataradious[i,2]).tolist();
            if len(includedset)>0:
                for k in range(len(includedset)):
                    in_id =includedset[k][0]
                    flag[leftindex[in_id]] = flag[i];

    belongto =np.zeros((len(dataradious),1));
    belongto[sortdataradious[:,0].tolist()] = flag;
    dataradious=dataradious.tolist()
    for k in range(len(dataradious)):
        dataradious[k].append(int(belongto[k,0]))
    dataradious=np.array(dataradious)    
    return [dataradious,pddist]

In [4]:
def calradious(i,ilabel,majlabel,minlabel,minENNdist,minenemykneigor,majENNdist,majenemykneigor):
    IR = len(majENNdist)/len(minENNdist);
    tha = 1.0/(1+np.exp(-(IR-1)));
    if ilabel == minlabel:
        nej = int(minenemykneigor[i][0]);
        dij = minENNdist[i][0];
        jlabel =  majlabel;
        ENN=int(majenemykneigor[nej][0])
        if ( i == ENN):
            dradius = dij*tha;
            return dradius ;
        else:
            dt = calradious(nej,jlabel,majlabel,minlabel,minENNdist,minenemykneigor,majENNdist,majenemykneigor);
            dradius = dij-dt;
            return dradius ;
    else:
        nej = int(majenemykneigor[i][0]);
        dij = majENNdist[i][0];
        jlabel =  minlabel;
        ENN=int(minenemykneigor[nej][0])
        if ( i == ENN):
            dradius = dij*(1-tha);
            return dradius ;
        else:
            dt = calradious(nej,jlabel,majlabel,minlabel,minENNdist,minenemykneigor,majENNdist,majenemykneigor);
            dradius = dij-dt;
            return dradius ;


In [5]:
def L2_dist(Min_data,Maj_data):
    x = Min_data;
    xsup = Maj_data;
    ps = np.dot(x,xsup.T); 
    [nps,pps]=ps.shape;
    normx =np.array([np.sum(x**2, axis=1)]).T;
    normxsup = np.sum(xsup**2, axis=1);
    A=np.matlib.repmat(normx,1,pps)
    B=np.matlib.repmat(normxsup.T,nps,1)
    ps = -2*ps + A + B; 
    E=np.sqrt(ps);
    return E
    


In [6]:
def kNNenemyPoint(Min_data,Maj_data):
    k_nNum = 1;
    NT = len(Min_data)
    NTNT = len(Maj_data)
    minENNdist=np.ones((NT,k_nNum))*math.inf;
    minenemykneigor=np.ones((NT,k_nNum))*math.inf;
    majENNdist= np.ones((NTNT,k_nNum))*math.inf;
    majenemykneigor =np.ones((NTNT,k_nNum))*math.inf;
    pd = L2_dist(Min_data,Maj_data);
    for i in range(NT):
        d=np.copy(pd[i,:])
        min_id=[]
        for k in range(k_nNum):
            tmp=np.min(d);
            index=np.argmin(d)
            d[index]=math.inf;
            min_id.append(index)
        minENNdist[i,:] = pd[i, min_id];
        minenemykneigor[i,:] =  min_id
    pd = pd.T
    majmindistmax = pd;
    for i in range(NTNT):   
        d=np.copy(pd[i,:])
        min_id=[];
        for k in range(k_nNum):
            tmp=np.min(d);
            index=np.argmin(d)
            d[index]=math.inf;
            min_id.append(index)
        majENNdist[i,:] = pd[i,min_id];
        majenemykneigor[i,:] = min_id
    return [minENNdist,minenemykneigor,majENNdist,majenemykneigor,majmindistmax]

# read dataset and split samples

In [8]:
#data_name='vehicle0.dat'
data_name='glass1.dat'
Size_data=846
C_maj=647
F_maj=18
C_min=199
F_min=18
head=23 # number of line without sample (header)

if data_name=='glass1.dat':
    Size_data=214
    C_maj=138
    F_maj=9
    C_min=76
    F_min=9
    head=14 # index of first data sample

if data_name=='ecoli1.dat':
    Size_data=336
    C_maj=259
    F_maj=7
    C_min=77
    F_min=7
    head=12 # index of first data sample
if data_name=='ecoli4.dat':
    Size_data=336
    C_maj=316
    F_maj=7
    C_min=20
    F_min=7
    head=12 # index of first data sample

if data_name=='glass4.dat':
    Size_data=214
    C_maj=201
    F_maj=9
    C_min=13
    F_min=9
    head=14 # index of first data sample
if data_name=='yeast-2_vs_8.dat':
    Size_data=482
    C_maj=462
    F_maj=8
    C_min=20
    F_min=8
    head=13 # index of first data sample

if data_name=='yeast4.dat':
    Size_data=1484
    C_maj=1433
    F_maj=8
    C_min=51
    F_min=8
    head=13 # index of first data sample
if data_name=='yeast-1-2-8-9_vs_7.dat':
    Size_data=947
    C_maj=917
    F_maj=8
    C_min=30
    F_min=8
    head=13 # index of first data sample

if data_name=='yeast5.dat':
    Size_data=1484
    C_maj=1440
    F_maj=8
    C_min=44
    F_min=8
    head=13 # index of first data sample 

if data_name=='abalone9-18.dat':
    Size_data=731
    C_maj=689
    F_maj=8
    C_min=42
    F_min=8
    head=13 # index of first data sample 

   
    
if data_name=='page-blocks-1-3_vs_4.dat':
    Size_data=472
    C_maj=444
    F_maj=10
    C_min=28
    F_min=10
    head=15 # index of first data sample 


if data_name=='glass-0-1-2-3_vs_4-5-6.dat':
    Size_data=214
    C_maj=163
    F_maj=9
    C_min=51
    F_min=9
    head=14 # index of first data sample  
    
if data_name=='glass0.dat':
    Size_data=214
    C_maj=144
    F_maj=9
    C_min=70
    F_min=9
    head=14 # index of first data sample     
  
if data_name=='vehicle3.dat':
    Size_data=846
    C_maj=634
    F_maj=18
    C_min=212
    F_min=18
    head=23 # index of first data sample     
if data_name=='ecoli2.dat':
    Size_data=336
    C_maj=284
    F_maj=7
    C_min=52
    F_min=7
    head=12 # index of first data sample 
if data_name=='yeast6.dat':
    Size_data=1484
    C_maj=1449
    F_maj=8
    C_min=35
    F_min=8
    head=13 # index of first data sample   
  
if data_name=='yeast-2_vs_4.dat':
    Size_data=514
    C_maj=463
    F_maj=8
    C_min=51
    F_min=8
    head=13 # index of first data sample    

if data_name=='yeast3.dat':
    Size_data=1484
    C_maj=1321
    F_maj=8
    C_min=163
    F_min=8
    head=13 # index of first data sample     

    
if data_name=='yeast-1-4-5-8_vs_7.dat':
    Size_data=693
    C_maj=663
    F_maj=8
    C_min=30
    F_min=8
    head=13 # index of first data sample     
if data_name=='segment0.dat':
    Size_data=2308
    C_maj=1979
    F_maj=19
    C_min=329
    F_min=19
    head=24 # index of first data sample 

if data_name=='yeast-0-5-6-7-9_vs_4.dat':
    Size_data=528
    C_maj=477
    F_maj=8
    C_min=51
    F_min=8
    head=13 # index of first data sample 

if data_name=='ecoli3.dat':
    Size_data=336
    C_maj=301
    F_maj=7
    C_min=35
    F_min=7
    head=12 # index of first data sample 
if data_name=='vowel0.dat':
    Size_data=988
    C_maj=898
    F_maj=13
    C_min=90
    F_min=13
    head=18 # index of first data sample 

if data_name=='glass2.dat':
    Size_data=214
    C_maj=197
    F_maj=9
    C_min=17
    F_min=9
    head=14 # index of first data sampl
if data_name=='glass5.dat':
    Size_data=214
    C_maj=205
    F_maj=9
    C_min=9
    F_min=9
    head=14 # index of first data sample 
    
if data_name=='glass6.dat':
    Size_data=214
    C_maj=185
    F_maj=9
    C_min=29
    F_min=9
    head=14 # index of first data sample 

if data_name=='new-thyroid1.dat':
    Size_data=215
    C_maj=180
    F_maj=5
    C_min=35
    F_min=5
    head=10 # index of first data sample
    
if data_name=='ecoli-0_vs_1.dat':
    Size_data=220
    C_maj=77
    F_maj=7
    C_min=143
    F_min=7
    head=12 # index of first data sample 
if data_name=='wisconsin.dat':
    Size_data=683
    C_maj=444
    F_maj=9
    C_min=239
    F_min=9
    head=14 # index of first data sample 
if data_name=='iris0.dat':
    Size_data=150
    C_maj=100
    F_maj=4
    C_min=50
    F_min=4
    head=9 # index of first data sample 
    
if data_name=='haberman.dat':
    Size_data=306
    C_maj=225
    F_maj=3
    C_min=81
    F_min=3
    head=8 # index of first data sample   
    
data_maj = np.zeros((C_maj,F_maj))
labels_maj = np.zeros((C_maj,))
data_min = np.zeros((C_min,F_min))
labels_min = np.zeros((C_min,))

f1=0
f2=0   
with open( data_name,'r') as file:
    a = file.readlines()
    print(a[0:head])
    for i in range(Size_data):
        a[i+head] = a[i+head].replace(" ","")
        b = a[i+head].split(',')
        print(i,b)
        if b[0]=='M':
            b[0]=0
        if b[0]=='F':
            b[0]=1
        if b[0]=='I':
            b[0]=2
        if b[F_maj].rfind("positive")!=(-1):
            b[F_maj] = -1
            data_min[f2,:] = np.array(b[:F_maj])
            labels_min[f2] = b[F_maj] 
            f2+=1
        elif b[F_maj].rfind("negative")!=(-1):
            b[F_maj] = 1
            data_maj[f1,:] = np.array(b[:F_maj])
            labels_maj[f1] = b[F_maj] 
            f1+=1
print(f1,f2)
'''Xtrn_maj, Xtst_maj, ytrn_maj, ytst_maj = train_test_split(data_maj, labels_maj, test_size=0.25, random_state=13)
Xtrn_min, Xtst_min, ytrn_min, ytst_min = train_test_split(data_min, labels_min, test_size=0.25, random_state=13)
scl = StandardScaler()
data_train=scl.fit_transform(np.vstack((Xtrn_min,Xtrn_maj)))
data_test = scl.transform(np.vstack((Xtst_min,Xtst_maj)))
features_min=data_train[0:len(Xtrn_min),:]
features_maj=data_train[len(Xtrn_min):,:] '''

['@relation glass1\n', '@attribute RI real [1.51115, 1.53393]\n', '@attribute Na real [10.73, 17.38]\n', '@attribute Mg real [0.0, 4.49]\n', '@attribute Al real [0.29, 3.5]\n', '@attribute Si real [69.81, 75.41]\n', '@attribute K real [0.0, 6.21]\n', '@attribute Ca real [5.43, 16.19]\n', '@attribute Ba real [0.0, 3.15]\n', '@attribute Fe real [0.0, 0.51]\n', '@attribute Class {positive, negative}\n', '@inputs RI, Na, Mg, Al, Si, K, Ca, Ba, Fe\n', '@outputs Class\n', '@data\n']
0 ['1.51588824', '12.87795', '3.43036', '1.40066', '73.282', '0.68931', '8.04468', '0.0', '0.1224', 'negative\n']
1 ['1.5176423', '12.9777', '3.53812', '1.21127', '73.002', '0.65205', '8.52888', '0.0', '0.0', 'negative\n']
2 ['1.52212996', '14.20795', '3.82099', '0.46976', '71.77', '0.11178', '9.5726', '0.0', '0.0', 'negative\n']
3 ['1.52222108', '13.21045', '3.7716', '0.79076', '71.9884', '0.13041', '10.2452', '0.0', '0.0', 'negative\n']
4 ['1.51755118', '13.39', '3.65935', '1.1888', '72.7892', '0.57132', '8.270

'Xtrn_maj, Xtst_maj, ytrn_maj, ytst_maj = train_test_split(data_maj, labels_maj, test_size=0.25, random_state=13)\nXtrn_min, Xtst_min, ytrn_min, ytst_min = train_test_split(data_min, labels_min, test_size=0.25, random_state=13)\nscl = StandardScaler()\ndata_train=scl.fit_transform(np.vstack((Xtrn_min,Xtrn_maj)))\ndata_test = scl.transform(np.vstack((Xtst_min,Xtst_maj)))\nfeatures_min=data_train[0:len(Xtrn_min),:]\nfeatures_maj=data_train[len(Xtrn_min):,:] '

# Hyperspheres covering minority data

In [9]:
scl = StandardScaler()
data_train=scl.fit_transform(np.vstack((data_min,data_maj)))
features_min=data_train[0:len(data_min),:]
features_maj=data_train[len(data_maj):,:]

In [10]:
dim=F_maj
IMratio = len(features_maj)/len(features_min)
[minENNdist,minenemykneigor,majENNdist,majenemykneigor,majmindistmax]=kNNenemyPoint(features_min,features_maj);
majlabel = 1;
minlabel = -1;
mindradious = [];
for i in range(len(features_min)):
    ilabel = minlabel;
    dradius = calradious(i,ilabel,majlabel,minlabel,minENNdist,minenemykneigor,majENNdist,majenemykneigor);
    mindradious.append([i,features_min[i,:],dradius]);
    #mindradious.append([i,dradius]);
mindradious=np.array(mindradious)
data = np.copy(features_min)
[mindradious,mindist] = unionSphere(data,mindradious);
uncoveredpoint_min = list(set(mindradious[:,-1]));
numofsphere_min = [];
for i in range(len(uncoveredpoint_min)):
        j = uncoveredpoint_min[i];
        numofsphere_min.append(np.sum(mindradious[:,-1]==j))
numofsphere_min=np.array(numofsphere_min)
finalmincenters = np.array(mindradious[uncoveredpoint_min,1].tolist());
finalminradius  = np.array(mindradious[uncoveredpoint_min,2].tolist()); 
uncoveredpoint_min

[1,
 4,
 7,
 8,
 10,
 11,
 12,
 13,
 16,
 18,
 19,
 27,
 30,
 31,
 32,
 35,
 36,
 38,
 41,
 43,
 44,
 45,
 47,
 48,
 49,
 51,
 53,
 59,
 60,
 61,
 63,
 64,
 67,
 69,
 70,
 71,
 72,
 73]

# Hyperspheres covering majority data

In [11]:
majdradious = [];
for i in range(len(features_maj)):
    ilabel = majlabel;
    dradius = calradious(i,ilabel,majlabel,minlabel,minENNdist,minenemykneigor,majENNdist,majenemykneigor);
    majdradious.append([i,features_maj[i,:],dradius]);
    #majdradious.append([i,dradius]);
majdradious=np.array(majdradious)
data = np.copy(features_maj)
[majdradious,majdist] = unionSphere(data,majdradious);
uncoveredpoint_maj = list(set(majdradious[:,-1]));
print(uncoveredpoint_maj)
numofsphere_maj = [];
for i in range(len(uncoveredpoint_maj)):
        j = uncoveredpoint_maj[i];
        numofsphere_maj.append(np.sum(majdradious[:,-1]==j))
numofsphere_maj=np.array(numofsphere_maj)
finalmajcenters = np.array(majdradious[uncoveredpoint_maj,1].tolist());
finalmajradius  = np.array(majdradious[uncoveredpoint_maj,2].tolist()); 


[1, 2, 4, 5, 7, 8, 10, 12, 13, 14, 16, 20, 21, 22, 24, 25, 26, 27, 28, 30, 33, 34, 36, 38, 39, 42, 44, 49, 50, 52, 55, 63, 66, 69, 72]


# Data complexity based AdaBoost 

In [12]:
def Oversampling(errors,oversamplesize,numofsphere_min,uncoveredpoint,finalminradius,finalmincenters ):
    dim=len(finalmincenters[0])
    if len(errors)==0:
        alpha = 1;
        beta = 1;
        pi = np.array(np.exp(-alpha*finalminradius));
        pj = np.array(np.exp(-beta*numofsphere_min));
        pi =np.array((pi+pj)/np.sum(pi+pj)) 
        suboversampesizes = np.ceil(pi*oversamplesize)
        generatedX = [];
        for i in range(len(uncoveredpoint)):
            X = gnerandsphere(finalmincenters[i,:],dim,int(suboversampesizes[i]),finalminradius[i]); 
            generatedX.append(X);
    else:
        generatedX = [];
        suboversampesizes=[]
        errors=np.array(errors)
        if np.sum(errors)!=0:
            errors=errors/np.sum(errors)
            suboversampesizes = np.ceil(errors*oversamplesize)
            for i in range(len(uncoveredpoint)):
                X = gnerandsphere(finalmincenters[i,:],dim,int(suboversampesizes[i]),finalminradius[i]); 
                generatedX.append(X);
    return generatedX,suboversampesizes
def removedsphere(center,N,X_maj,D_maj):
    X_maj=np.vstack(X_maj)
    #pd=L2_dist(np.zeros((1,len(center)))+center,X_maj)
    #idx=np.argpartition(pd,N)[0,0:N]
    idx=np.argsort(D_maj)[0:N]
    X_maj=np.delete(X_maj, idx,0)
    D_maj=np.delete(D_maj, idx,0)
    #print(X_maj.shape)
    return X_maj,D_maj
    
def Undersampling(X_maj_Hyperspheres,D_maj_H,errors,undersamplesize,numofsphere_maj,uncoveredpoint_maj,finalmajradius,finalmajcenters ):
    if len(errors)==0:
        dim=len(finalmajcenters[0])
        alpha = 1;
        beta = 1;
        pi = 1-np.array(np.exp(-alpha*finalmajradius));
        pj = 1-np.array(np.exp(-beta*numofsphere_maj));
        pi =np.array((pi+pj)/np.sum(pi+pj)) 
        subundersampesizes = np.ceil(pi*undersamplesize)
        remainingX = [];
        remainingD=[]
        for i in range(len(uncoveredpoint_maj)):
            if len(X_maj_Hyperspheres[i])>2:
                X,D = removedsphere(finalmajcenters[i,:],int(subundersampesizes[i]),X_maj_Hyperspheres[i],D_maj_H[i]); 
                remainingX.append(X);
                remainingD.append(D);
            else:
                #print("   mmm", X_maj_Hyperspheres[i])
                remainingX.append(X_maj_Hyperspheres[i])
                remainingD.append(D_maj_H[i])
                
    else:
        subundersampesizes=[]
        remainingX = [];
        remainingD=[]
        errors=1-np.array(errors)
        if np.sum(errors)!=0:
            errors=errors/np.sum(errors)
            subundersampesizes = np.ceil(errors*undersamplesize)
            for i in range(len(uncoveredpoint_maj)):
                if len(X_maj_Hyperspheres[i])>2:
                    if len(X_maj_Hyperspheres[i])>=int(subundersampesizes[i]):
                        N=int(subundersampesizes[i])
                    else:
                        N=len(X_maj_Hyperspheres[i])-1
                    X,D = removedsphere(finalmajcenters[i,:],N,X_maj_Hyperspheres[i],D_maj_H[i]); 
                    remainingX.append(X);
                    remainingD.append(D);
                else:
                    #print("   mmm", X_maj_Hyperspheres[i])
                    remainingX.append(X_maj_Hyperspheres[i])
                    remainingD.append(D_maj_H[i])
    return remainingX, remainingD, subundersampesizes



In [13]:
def fit_boosting(X_maj, X_min, y_maj, y_min,n_estimators,majdradious,mindradious, oversamplesize, undersamplesize,train_index):
    #determine number of Hyperspheres in dataset
    uncoveredpoint_min = list(set(mindradious[:,-1]));
    uncoveredpoint_maj = list(set(majdradious[:,-1]));
    num_Hyperspheres=len(uncoveredpoint_min)+len(uncoveredpoint_maj)
    #initial weights in majority Class
    numofsphere_maj = [];
    D_maj_H=[]
    X_maj_Hyperspheres=[]
    for i in range(len(uncoveredpoint_maj)):
        count_sam=0
        sub_samples=[]
        j = uncoveredpoint_maj[i];
        k=majdradious[majdradious[:,-1]==j][:,0]+f2
        sub_maj=majdradious[majdradious[:,-1]==j][:,1]
        for m in range(len(sub_maj)):
            if k[m] in train_index:
                count_sam+=1
                sub_samples.append(sub_maj[m].tolist())
        if count_sam==0:
            n_samples=np.sum(count_sam)
            numofsphere_maj.append(n_samples)
        if count_sam>0:
            X_maj_Hyperspheres.append(np.array( sub_samples))
            n_samples=np.sum(count_sam)
            numofsphere_maj.append(n_samples)
            D_H=np.ones((n_samples, ))/(n_samples*num_Hyperspheres)
            D_maj_H.append(D_H)
    numofsphere_maj=np.array(numofsphere_maj)
    finalmajcenters = np.array(majdradious[uncoveredpoint_maj,1].tolist());
    finalmajradius  = np.array(majdradious[uncoveredpoint_maj,2].tolist());
    # initial weights in Minority Class
    numofsphere_min = [];
    D_min_H=[]
    X_min_Hyperspheres=[]
    for i in range(len(uncoveredpoint_min)):
        count_sam=0
        sub_samples=[]
        j = uncoveredpoint_min[i];
        k=majdradious[mindradious[:,-1]==j][:,0]
        sub_min=mindradious[mindradious[:,-1]==j][:,1]
        for m in range(len(sub_min)):
            if k[m] in train_index:
                count_sam+=1
                sub_samples.append(sub_min[m].tolist())
        if count_sam==0:
            n_samples=count_sam
            numofsphere_min.append(n_samples)
        if count_sam>0:
            X_min_Hyperspheres.append(np.array( sub_samples))
            n_samples=np.sum(count_sam)
            numofsphere_min.append(n_samples)
            D_H=np.ones((n_samples, ))/(n_samples*num_Hyperspheres)
            D_min_H.append(D_H)
    numofsphere_min=np.array(numofsphere_min)
    finalmincenters = np.array(mindradious[uncoveredpoint_min,1].tolist());
    finalminradius  = np.array(mindradious[uncoveredpoint_min,2].tolist());
    
    n_samples_maj, n_features = X_maj.shape
    n_samples_min, n_features = X_min.shape
    estimators = []
    errors_maj=[]
    errors_min=[]
    a_maj=[] 
    a_min=[]
    D_syns_steps=[]
    for t in range(n_estimators):
        print("t ", t)
        #Oversampling
        if t==0:
            X_syns,suboversampesizes= Oversampling(errors_min,oversamplesize,numofsphere_min,uncoveredpoint_min,finalminradius,finalmincenters)
            X_syns=np.vstack(X_syns)
            X_min=np.vstack((X_min,X_syns))
            D_syns=np.ones((X_syns.shape[0], ))/X_syns.shape[0]
            D_syns_steps.append(D_syns)
            D_syns=np.hstack(D_syns_steps)
        else:
            D_syns=np.hstack(D_syns_steps)
        #Undersampling
        '''if len(X_min)<len(X_maj):
                X_Under,D_Under,subundersampesizes=Undersampling(X_maj_Hyperspheres,D_maj_H,errors_maj,undersamplesize,numofsphere_maj,uncoveredpoint_maj,finalmajradius,finalmajcenters )
                X_maj=np.vstack(X_Under)
                D_maj=np.hstack(D_Under)
        
        else:'''
        X_maj=np.vstack(X_maj_Hyperspheres)
        D_maj=np.hstack(D_maj_H)
        X=np.vstack((X_maj,X_min))

        
        y_min = np.full(X_min.shape[0], fill_value=y_min[0], dtype=np.int64)
        y_maj = np.full(X_maj.shape[0], fill_value=y_maj[0], dtype=np.int64)
        y=np.hstack((y_maj,y_min))
        
        print(y)
        
        D_min=np.hstack(D_min_H)
        D=np.hstack((D_maj,D_min,D_syns))
        D=D/np.sum(D)
        
        h = DecisionTreeClassifier(max_depth=5)  
        h.fit(X, y, sample_weight=D) 
        
        errors_maj=[]
        errors_min=[]
        a_maj=[] 
        a_min=[]
        print("len",len(X_maj), len(X_min))
        for i in range(len(X_min_Hyperspheres)):
            ypred_min=h.predict(X_min_Hyperspheres[i])
            y_min=np.full(X_min_Hyperspheres[i].shape[0], fill_value=y_min[0], dtype=np.int64)
            e_min = 1 - accuracy_score(y_min, ypred_min)
            if e_min==0.0:
                e_min=0.000000000001
            if e_min==1.0:
                e_min=0.9999
            errors_min.append(e_min)
            a= 0.5 * np.log((1 - e_min) / e_min)
            a_min.append(a) 
            m_min = (y_min == ypred_min) * 1 + (y_min != ypred_min) * -1 
            D_min_H[i] *= np.exp(-a * m_min)
                              
          
        for i in range(len(X_maj_Hyperspheres)):
            ypred_maj=h.predict(X_maj_Hyperspheres[i]) 
            y_maj=np.full(X_maj_Hyperspheres[i].shape[0], fill_value=y_maj[0], dtype=np.int64)
            e_maj = 1 - accuracy_score(y_maj, ypred_maj)
            if e_maj==0.0:
                e_maj=0.000000000001
            if e_maj==1.0:
                e_maj=0.9999
            
            errors_maj.append(e_maj)
            a = 0.5 * np.log((1 - e_maj) / e_maj) 
            a_maj.append(a) 
            m_maj = (y_maj == ypred_maj) * 1 + (y_maj != ypred_maj) * -1 
            D_maj_H[i] *= np.exp(-a * m_maj)
        a=np.sum(np.array(a_maj))+np.sum(np.array(a_min))
        estimators.append((a, h))  

    return estimators

# Predict

In [14]:
def predict_boosting(X, estimators):
    pred = np.zeros((X.shape[0], ))
    y_proba = np.zeros((X.shape[0], ))

    for a, h in estimators:
        pred += a * h.predict(X)
        y_proba+= h.predict_proba(X)[:, 1] 

    y = np.sign(pred)

    return y , (y_proba/len(estimators))

# report Metrics

In [15]:
def split_min_maj(data, labels):
    d_min=[]
    d_maj=[]
    l_min=[]
    l_maj=[]
    for i in range(len(data)):
        if labels[i]==-1:
            d_min.append(data[i])
            l_min.append(labels[i])
        else:
            d_maj.append(data[i]) 
            l_maj.append(labels[i])
    return np.array(d_min),np.array(l_min),np.array(d_maj),np.array(l_maj)
from sklearn.model_selection import KFold
from imblearn.metrics import geometric_mean_score
data=np.vstack((data_min,data_maj))
labels=np.hstack((labels_min,labels_maj))
kf = KFold(n_splits=5,random_state=76,shuffle=True) # 5fold cross validation
k_g_mean=[]
k_roc=[]
k_PR=[]
k_imb_g=[]
scl = StandardScaler()
for train_index, test_index in kf.split(data):

        data_train=scl.fit_transform(data[train_index])
        data_test=scl.transform(data[test_index])
        ytrn=labels[train_index]
        ytst=labels[test_index]
        Xtrn_min,  ytrn_min, Xtrn_maj, ytrn_maj=split_min_maj(data_train, ytrn)
        complexitymeasure = 1;
        oversamplesize = math.ceil(complexitymeasure*(len(features_maj)-len(features_min)));
        print(oversamplesize)
        over_size=int(0.8*(oversamplesize))
        under_size=int(0.2*(oversamplesize))
        estimators = fit_boosting(Xtrn_maj,Xtrn_min, ytrn_maj, ytrn_min,10,majdradious,mindradious,over_size,under_size,train_index)
        Xtst_min,  ytst_min, Xtst_maj, ytst_maj=split_min_maj(data_test, ytst)
        Xtst_maj = scl.transform(Xtst_maj)
        print("Xtst_maj ", Xtst_maj )
        Xtst_min = scl.transform(Xtst_min)
        ypred_maj,_ = predict_boosting(Xtst_maj, estimators)
        ypred_min,_ = predict_boosting(Xtst_min, estimators)
        from sklearn.metrics import accuracy_score
        #tst_err = 1 - accuracy_score(ytst, ypred)
        #tst_err
        print("negative: ", accuracy_score(ytst_maj, ypred_maj))
        print("positive: ", accuracy_score(ytst_min, ypred_min))
        y_test=np.hstack((ytst_min,ytst_maj))
        predictions, y_proba=predict_boosting(data_test, estimators)
        print("Confusion Matrix:")
        print(confusion_matrix(y_test, predictions))
        print()
        print("Classification Report")
        print(classification_report(y_test, predictions))
        
        roc_auc= roc_auc_score(y_test, y_proba)
        precision, recall, thresholds = precision_recall_curve(y_test, y_proba)
        PR_auc_score = auc(recall, precision)
        g_mean = 1.0
        for label in np.unique(y_test):
            idx = (y_test== label)
            ytest=np.array(y_test)
            preds=np.array(predictions)
            g_mean *= accuracy_score(ytest[idx], predictions[idx])
        g_mean = np.sqrt(g_mean)
        g=geometric_mean_score(y_test, predictions,average='macro')
        k_imb_g.append(g)
        print( g_mean,"***********",g)
        k_g_mean.append(g_mean)
        k_roc.append(roc_auc)
        k_PR.append(PR_auc_score)

    

print("G_means: ", k_g_mean, "Mean:",np.array(k_g_mean).mean())
print("G_means: ", k_imb_g, "Mean:",np.array(k_imb_g).mean())
print("roc_auc: ", k_roc, "Mean:",np.array(k_roc).mean())
print("PR_auc_score: ", k_PR, "Mean:",np.array(k_PR).mean())

0
t  0
[ 1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1
  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1
  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1 -1 -1 -1 -1 -1 -1 -1 -1
 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1
 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1
 -1 -1 -1]
len 64 59
t  1
[ 1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1
  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1
  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1 -1 -1 -1 -1 -1 -1 -1 -1
 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1
 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1
 -1 -1 -1]
len 64 59
t  2
[ 1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1
  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1
  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1 -1 -1 -1 -1 -1 -

[ 1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1
  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1
  1  1  1  1  1  1  1  1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1
 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1
 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1]
len 56 64
t  3
[ 1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1
  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1
  1  1  1  1  1  1  1  1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1
 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1
 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1]
len 56 64
t  4
[ 1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1
  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1
  1  1  1  1  1  1  1  1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1
 -1 -1 -1 -1 -1 -1 

len 65 55
t  4
[ 1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1
  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1
  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1 -1 -1 -1 -1 -1 -1 -1
 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1
 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1]
len 65 55
t  5
[ 1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1
  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1
  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1 -1 -1 -1 -1 -1 -1 -1
 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1
 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1]
len 65 55
t  6
[ 1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1
  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1
  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1 -1 -1 -1 -1 -1 -1 -1
 -1 

t  6
[ 1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1
  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1
  1  1  1  1  1  1  1  1  1  1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1
 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1
 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1
 -1 -1 -1 -1]
len 58 66
t  7
[ 1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1
  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1
  1  1  1  1  1  1  1  1  1  1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1
 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1
 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1
 -1 -1 -1 -1]
len 58 66
t  8
[ 1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1
  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1
  1  1  1  1  1  1  1  1  1  1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 

t  7
[ 1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1
  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1
  1  1  1  1  1  1  1  1  1  1  1  1  1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1
 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1
 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1
 -1]
len 61 60
t  8
[ 1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1
  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1
  1  1  1  1  1  1  1  1  1  1  1  1  1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1
 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1
 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1
 -1]
len 61 60
t  9
[ 1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1
  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1
  1  1  1  1  1  1  1  1  1  1  1  1  1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1
 -1 -1

In [ ]:
complexitymeasure = 1;
oversamplesize = math.ceil(complexitymeasure*(len(features_maj)-len(features_min)));
print(oversamplesize)
over_size=int(0.8*(oversamplesize))
under_size=int(0.2*(oversamplesize))
estimators = fit_boosting(Xtrn_maj,Xtrn_min, ytrn_maj, ytrn_min,30,majdradious,mindradious,over_size,under_size)
Xtst_maj = scl.transform(Xtst_maj)
print("Xtst_maj ", Xtst_maj )
Xtst_min = scl.transform(Xtst_min)
ypred_maj,_ = predict_boosting(Xtst_maj, estimators)
ypred_min,_ = predict_boosting(Xtst_min, estimators)
from sklearn.metrics import accuracy_score
#tst_err = 1 - accuracy_score(ytst, ypred)
#tst_err
print("negative: ", accuracy_score(ytst_maj, ypred_maj))
print("positive: ", accuracy_score(ytst_min, ypred_min))
y_test=np.hstack((ytst_min,ytst_maj))
predictions, y_proba=predict_boosting(data_test, estimators)
print("Confusion Matrix:")
print(confusion_matrix(y_test, predictions))
print()
print("Classification Report")
print(classification_report(y_test, predictions))
roc_auc= roc_auc_score(y_test, y_proba)
precision, recall, thresholds = precision_recall_curve(y_test, y_proba)
PR_auc_score = auc(recall, precision)
g_mean = 1.0
for label in np.unique(y_test):
    idx = (y_test== label)
    ytest=np.array(y_test)
    preds=np.array(predictions)
    g_mean *= accuracy_score(ytest[idx], predictions[idx])
g_mean = np.sqrt(g_mean)
print("G_means: ", g_mean)
print("ROC AUC: ", roc_auc)
print("PR AUC: ", PR_auc_score)
predictions

In [ ]:

complexitymeasure = len(uncoveredpoint)/len(features_min)
complexitymeasure = 1;
oversamplesize = math.ceil(complexitymeasure*(len(features_maj)-len(features_min)));
print(oversamplesize)
alpha = 1;
beta = 1;
pi = np.array(np.exp(-alpha*finalminradius));
pj = np.array(np.exp(-beta*numofsphere));
pi =np.array((pi+pj)/np.sum(pi+pj)) 
suboversampesizes = np.ceil(pi*oversamplesize)
generatedX = [];
for i in range(len(uncoveredpoint)):
        X = gnerandsphere(finalmincenters[i,:],dim,int(suboversampesizes[i]),finalminradius[i]); 
        generatedX.append([X]);



In [ ]:
a=np.array([1,2,3,4])
b=np.array([[1,2,3,-4],[-5,6,7,8]])
np.delete(a,2,0)

In [ ]:
gclass6:20, -10

In [ ]:

new-thyroid1.dat:30,+20
negative:  0.9333333333333333
positive:  1.0
Confusion Matrix:
[[ 9  0]
 [ 3 42]]

Classification Report
              precision    recall  f1-score   support

        -1.0       0.75      1.00      0.86         9
         1.0       1.00      0.93      0.97        45

    accuracy                           0.94        54
   macro avg       0.88      0.97      0.91        54
weighted avg       0.96      0.94      0.95        54

G_means:  0.966091783079295

In [ ]:
glass5:10,0
negative:  0.9807692307692307
positive:  1.0
Confusion Matrix:
[[ 3  0]
 [ 1 51]]

Classification Report
              precision    recall  f1-score   support

        -1.0       0.75      1.00      0.86         3
         1.0       1.00      0.98      0.99        52

    accuracy                           0.98        55
   macro avg       0.88      0.99      0.92        55
weighted avg       0.99      0.98      0.98        55

G_means:  0.9903379376602871

In [ ]:
glass2: 10,0

negative:  0.66
positive:  0.8
Confusion Matrix:
[[ 4  1]
 [17 33]]

Classification Report
              precision    recall  f1-score   support

        -1.0       0.19      0.80      0.31         5
         1.0       0.97      0.66      0.79        50

    accuracy                           0.67        55
   macro avg       0.58      0.73      0.55        55
weighted avg       0.90      0.67      0.74        55

G_means:  0.726636084983398

In [ ]:
wisconsin.dat: 10,-20
G_means:  0.9736004816647832
ROC AUC:  0.9933933933933934
PR AUC:  0.9971202505839244

In [ ]:
yeast-1-4-5-8_vs_7.dat: 

In [ ]:
glass6:
G_means:  0.9152955574276588
ROC AUC:  0.9707446808510638
PR AUC:  0.9945827004903716

In [ ]:
haberman: 20, -50
G_means:  0.6069769786668839
ROC AUC:  0.7940685045948204
PR AUC:  0.9270232564321046

In [ ]:
glass0:
G_means:  0.7817359599705717
ROC AUC:  0.8472222222222222
PR AUC:  0.9313027096360431

In [ ]:
glass-0-1-2-3_vs_4-5-6.dat:10,0
G_means:  0.9106416928094863
ROC AUC:  0.975609756097561
PR AUC:  0.9926768915208666   

In [ ]:
eciol3:10,(5,5)
G_means:  0.910465468000326
ROC AUC:  0.945906432748538
PR AUC:  0.9938305857747383

In [ ]:
page-blocks-1-3_vs_4.dat:
G_means:  0.995485304256668
ROC AUC:  1.0
PR AUC:  1.0

In [ ]:
ecoli1
G_means:  0.8133501469468494
ROC AUC:  0.9534615384615385
PR AUC:  0.987351382049489

In [ ]:

G_means:  0.721568539381252
ROC AUC:  0.8523109886746251
PR AUC:  0.9944298986273847

In [ ]:
G_means:  0.724904183060784
ROC AUC:  0.6946925906463478
PR AUC:  0.9681745411642522

In [ ]:
from sklearn.model_selection import KFold
kf = KFold(n_splits=5,random_state=76,shuffle=True) # 5fold cross validation
d_train_maj=[]
d_test_maj=[]
d_train_min=[]
d_test_min=[]
l_train_maj=[]
l_test_maj=[]
l_train_min=[]
l_test_min=[]
d_min=np.copy(data_min)
d_maj=np.copy(data_maj)
k_g_mean=[]
k_roc=[]
k_PR=[]
for train_index, test_index in kf.split(d_maj):
    d_train_maj.append(d_maj[train_index])
    d_test_maj.append(d_maj[test_index])
    l_train_maj.append(labels_maj[train_index])
    l_test_maj.append(labels_maj[test_index])

for train_index, test_index in kf.split(d_min):
    d_train_min.append(d_min[train_index])
    d_test_min.append(d_min[test_index]) 
    l_train_min.append(labels_min[train_index])
    l_test_min.append(labels_min[test_index])
    
for i in range(len(d_train_maj)):
    Xtrn_min=d_train_min[i]
    Xtrn_maj=d_train_maj[i]
    Xtst_min=d_test_min[i]
    Xtst_maj=d_test_maj[i]
    ytrn_maj=l_train_maj[i]
    ytst_maj=l_test_maj[i]
    ytrn_min=l_train_min[i]
    ytst_min=l_test_min[i]
    scl = StandardScaler()
    data_train=scl.fit_transform(np.vstack((Xtrn_min,Xtrn_maj)))
    data_test = scl.transform(np.vstack((Xtst_min,Xtst_maj)))
    features_min=data_train[0:len(Xtrn_min),:]
    features_maj=data_train[len(Xtrn_min):,:]
    dim=F_maj
    IMratio = len(features_maj)/len(features_min)
    [minENNdist,minenemykneigor,majENNdist,majenemykneigor,majmindistmax]=kNNenemyPoint(features_min,features_maj);
    majlabel = 1;
    minlabel = -1;
    mindradious = [];
    for i in range(len(features_min)):
        ilabel = minlabel;
        dradius = calradious(i,ilabel,majlabel,minlabel,minENNdist,minenemykneigor,majENNdist,majenemykneigor);
        mindradious.append([i,features_min[i,:],dradius]);
        #mindradious.append([i,dradius]);
    mindradious=np.array(mindradious)
    data = np.copy(features_min)
    [mindradious,mindist] = unionSphere(data,mindradious);
    uncoveredpoint_min = list(set(mindradious[:,-1]));
    numofsphere_min = [];
    for i in range(len(uncoveredpoint_min)):
            j = uncoveredpoint_min[i];
            numofsphere_min.append(np.sum(mindradious[:,-1]==j))
    numofsphere_min=np.array(numofsphere_min)
    finalmincenters = np.array(mindradious[uncoveredpoint_min,1].tolist());
    finalminradius  = np.array(mindradious[uncoveredpoint_min,2].tolist()); 
    majdradious = [];
    for i in range(len(features_maj)):
        ilabel = majlabel;
        dradius = calradious(i,ilabel,majlabel,minlabel,minENNdist,minenemykneigor,majENNdist,majenemykneigor);
        majdradious.append([i,features_maj[i,:],dradius]);
        #majdradious.append([i,dradius]);
    majdradious=np.array(majdradious)
    data = np.copy(features_maj)
    [majdradious,majdist] = unionSphere(data,majdradious);
    uncoveredpoint_maj = list(set(majdradious[:,-1]));
    print(uncoveredpoint_maj)
    numofsphere_maj = [];
    for i in range(len(uncoveredpoint_maj)):
            j = uncoveredpoint_maj[i];
            numofsphere_maj.append(np.sum(majdradious[:,-1]==j))
    numofsphere_maj=np.array(numofsphere_maj)
    finalmajcenters = np.array(majdradious[uncoveredpoint_maj,1].tolist());
    finalmajradius  = np.array(majdradious[uncoveredpoint_maj,2].tolist()); 
    complexitymeasure = 1;
    oversamplesize = math.ceil(complexitymeasure*(len(features_maj)-len(features_min)));
    #print(oversamplesize)
    estimators = fit_boosting(Xtrn_maj,Xtrn_min, ytrn_maj, ytrn_min,10,majdradious,mindradious, (oversamplesize//20), (oversamplesize//20))
    Xtst_maj = scl.transform(Xtst_maj)
    print("Xtst_maj ", Xtst_maj )
    Xtst_min = scl.transform(Xtst_min)
    ypred_maj,_ = predict_boosting(Xtst_maj, estimators)
    ypred_min,_ = predict_boosting(Xtst_min, estimators)
    from sklearn.metrics import accuracy_score
    #tst_err = 1 - accuracy_score(ytst, ypred)
    #tst_err
    print("negative: ", accuracy_score(ytst_maj, ypred_maj))
    print("positive: ", accuracy_score(ytst_min, ypred_min))
    y_test=np.hstack((ytst_min,ytst_maj))
    predictions, y_proba=predict_boosting(data_test, estimators)
    print("Confusion Matrix:")
    print(confusion_matrix(y_test, predictions))
    print()
    print("Classification Report")
    print(classification_report(y_test, predictions))
    roc_auc= roc_auc_score(y_test, y_proba)
    precision, recall, thresholds = precision_recall_curve(y_test, y_proba)
    PR_auc_score = auc(recall, precision)
    g_mean = 1.0
    k_roc.append(roc_auc)
    k_PR.append(PR_auc_score)
    for label in np.unique(y_test):
        idx = (y_test== label)
        ytest=np.array(y_test)
        preds=np.array(predictions)
        g_mean *= accuracy_score(ytest[idx], predictions[idx])
    g_mean = np.sqrt(g_mean)
    k_g_mean.append(g_mean)
    print("G_means: ", g_mean)
    print("ROC AUC: ", roc_auc)
    print("PR AUC: ", PR_auc_score)
    #predictions





print("G_means: ", k_g_mean, "Mean:",np.array(k_g_mean).mean())
print("roc_auc: ", k_roc, "Mean:",np.array(k_roc).mean())
print("PR_auc_score: ", k_PR, "Mean:",np.array(k_PR).mean())

In [ ]:
x=[1,2,3,4]
print([1,2] in x)